In [1]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

print("Hello to Deep Learning")

Hello to Deep Learning


### Installing some main torch library and other essential tools.
There are a number of imports for torch. (These are not included with ISLP, so must be installed separately.) All of them are used to specify sequentially-structured networks.

In [2]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError,R2Score)
from torchinfo import summary

 The package pytorch_lightning is a somewhat higher-level interface to torch that simplifies the specification and fitting of models by reducing the amount of boilerplate code needed (compared to using torch alone).

In order to reproduce results we use seed_everything(). We will also instruct torch to use deterministic algorithms where possible.

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)
#We will use several datasets shipped with torchvision for our examples
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)
# The glob() function from the glob module is used to find all files matching wildcard characters
from glob import glob
import json

Seed set to 0


In [4]:
Hitters = load_data('Hitters').dropna()
print(Hitters.shape)
n = Hitters.shape[0]

(263, 20)


We will fit two linear models (least squares and lasso) and compare their performance to that of a neural network. For this comparison we will use mean absolute error on a validation dataset. In tools like ModelSpec (from the ISLP library), the design matrix allows you to specify transformations, interaction terms, and polynomial expansions of features.

It also allows machine learning models to understand and process the input data as well handle categorical variables (using one-hot encoding).Overall, fit_transform() in ModelSpec automates the creation of a design matrix, preparing data for machine learning models by fitting and transforming it in one step​.

In [5]:
model = MS(Hitters.columns.drop('Salary'),intercept=False)
X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary'].to_numpy()
model

ModelSpec(intercept=False,
          terms=Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'NewLeague'],
      dtype='object'))

The to_numpy() method above converts pandas data frames or series to numpy arrays. We do this because we will need to use sklearn to fit the lasso model, and it requires this conversion. We now split the data into test and training, fixing the random state used by sklearn to do the split.

In [6]:
(X_train, 
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)
# X_train.shape
# X_test.shape
# Y_train.shape

### Linear Models

We fit the linear model and evaluate the test error directly.   

In [7]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat_test = hit_lm.predict(X_test)
np.abs(Yhat_test - Y_test).mean()

259.71528833146294

Next we fit the lasso using sklearn. We are using mean absolute error to select and evaluate a model, rather than mean squared error. 
Pipeline is a scikit-learn class used to chain multiple steps together. In this case, it chains standardization and Lasso regression into a single workflow.

In [8]:
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True, max_iter=30000)
standard_lasso = Pipeline(steps=[('scaler', scaler),
                                 ('lasso', lasso)])

X_s = scaler.fit_transform(X_train)
n = X_s.shape[0]
lam_max = np.fabs(X_s.T.dot(Y_train - Y_train.mean())).max() / n
param_grid = {'lasso__alpha': np.exp(np.linspace(0, np.log(0.01), 100))
             * lam_max}
# param_grid
# lam_max

lam_max is the value of the regularization parameter (lambda) beyond which all coefficients in the Lasso regression model would be shrunk to zero. It's essentially the highest level of regularization that would still allow some coefficients to remain non-zero.

We also now perform cross validation using this sequence of tuning regularization parameter (lambda) values.The 'param_grid' can be used in conjunction with GridSearchCV to find the optimal value of lambda for Lasso regression:

In [9]:
cv = KFold(10,
           shuffle=True,
           random_state=1)
grid = GridSearchCV(standard_lasso,
                    param_grid,
                    cv=cv,
                    scoring='neg_mean_absolute_error')
grid.fit(X_train, Y_train);
# grid

We extract the lasso model with best cross-validated mean absolute error, and evaluate its performance on X_test and Y_test, which were not used in cross-validation.

In [10]:
trained_lasso = grid.best_estimator_
Yhat_test = trained_lasso.predict(X_test)
np.fabs(Yhat_test - Y_test).mean()

235.6754837478029

### Specifying a Neural Network: Classes and Inheritance
To fit the neural network, we first set up a model structure that describes the network. Doing so requires us to define new classes specific to the model we wish to fit. Typically this is done in pytorch by sub-classing a generic representation of a network, which is the approach we take here.

In [11]:
#specifying network architecture
class HittersModel(nn.Module): #this class inherits from nn.module

    #create objects  
    def __init__(self,input_size):
        super(HittersModel,self).__init__()
        self.flatten = nn.Flatten()
        #nn.Sequential is a container that holds layers in sequence. 
        #The input data flows through these layers one by one.
        self.sequential = nn.Sequential(
            nn.Linear(input_size,50), # Fully connected layer: input -> 50 neurons
            nn.ReLU(), 
            nn.Dropout(0.4), # Dropout with a probability of 0.4
            nn.Linear(50,1) # Fully connected layer: 50 -> output 1 neuron
        )

    def forward(self,x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))

This defines a simple feedforward neural network (single hidden layer) with 50 neurons in the hidden layer and a ReLU activation function, followed by dropout and a final output layer. The class above inherits from nn.module base class.

The __init__ method creates instances of the hitters model class and has two model attributes flatten and sequential which are used in the forward method. 'super()' allows subclasses (i.e. HittersModel) to access methods of the class they inherit from. 

In the sequential attribute, the input data flows through all the layers (along with applied functions) we define one by one such as the Rectified Linear Activation Function,40% dropout layer and hidden layer(s) (here size -> 50 and features are 19).All these layers are mapped to one another.

In [12]:
hit_model = HittersModel(X.shape[1])
hit_model

HittersModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (sequential): Sequential(
    (0): Linear(in_features=19, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=50, out_features=1, bias=True)
  )
)

The package torchinfo provides a summary() function that neatly summarizes this information. We specify the size of the input and see the size of each tensor as it passes through layers of the network.

In [13]:
summary(hit_model, 
        input_size=X_train.shape,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
HittersModel                             [175, 19]                 [175]                     --
├─Flatten: 1-1                           [175, 19]                 [175, 19]                 --
├─Sequential: 1-2                        [175, 19]                 [175, 1]                  --
│    └─Linear: 2-1                       [175, 19]                 [175, 50]                 1,000
│    └─ReLU: 2-2                         [175, 50]                 [175, 50]                 --
│    └─Dropout: 2-3                      [175, 50]                 [175, 50]                 --
│    └─Linear: 2-4                       [175, 50]                 [175, 1]                  51
Total params: 1,051
Trainable params: 1,051
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.18
Input size (MB): 0.01
Forward/backward pass size (MB): 0.07
Params size (MB): 0.00
Estimated Total Size (MB): 0.09

Now we wish to see the final results (i.e. the error metrics) of this neural network and should be expecting some good answers (compared to the previous models) as there is quite a lot of parameters.

Since the data is a bit different, i.e. the datatypes are not tensors which are the basic DT in torch, we need to convert it to a more accesible form.As usual we again create a test and training dataset split, this time using the TensorDataset function.


In [14]:
X_train_t = torch.tensor(X_train.astype(np.float32)) # numpy array(s)
Y_train_t = torch.tensor(Y_train.astype(np.float32))
hit_train = TensorDataset(X_train_t, Y_train_t)

#test data
X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.float32))
hit_test = TensorDataset(X_test_t, Y_test_t)

max_num_workers = rec_num_workers()

Finally, this dataset is passed to a DataLoader() which ultimately passes data into our network. While this may seem like a lot of overhead, this structure is helpful for more complex tasks where data may live on different machines, or where data must be passed to a GPU. We provide a helper function SimpleDataModule() provided in ISLP to make this task easier for standard usage.

We now use this to tell torch how to read in the data for Stochastic gradient descent.We provide the training and test data sets as well as the batch size (of 32) and a validation set (test set only in this case).The parameter 'num_workers' indicates how many processes we will use for loading the data.

By using the SimpleModule.regression() method, we indicate that we will use squared-error loss.We have also asked for mean absolute error to be tracked as well in the metrics that are logged.

The whole process might feel cumbersome but tends to repeat itself and can be redone after the steps are understood.

In [19]:
hit_dm = SimpleDataModule(hit_train,
                          hit_test,
                          batch_size=32,
                          num_workers=min(4, max_num_workers),
                          validation=hit_test)

#prediction and measuring squared error loss
hit_module = SimpleModule.regression(hit_model,metrics={'mae':MeanAbsoluteError()})

# We can log our results via CSVLogger(), which in this case stores the results in a CSV file within a directory logs/hitters
hit_logger = CSVLogger('logs', name='hitters')

Now we are finally ready to train our model and put everything together to run stochastic gradient descent.We provide some logging details and maximum no of epochs information in the Trainer method (object from pytorch_lightning).The callback argument uses the helper function ErrorTracker() to track the error/loss (at every epoch) in our logs and we use this to make plots as we go along the epochs. 

In [20]:
hit_trainer = Trainer(deterministic=True,
                      max_epochs=50,
                      log_every_n_steps=5,
                      logger=hit_logger,
                      callbacks=[ErrorTracker()])

hit_trainer.fit(hit_module,datamodule=hit_dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | HittersModel | 1.1 K  | train
1 | loss  | MSELoss      | 0      | train
-----------------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 48.94it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 40.78it/s, v_num=0]


After having fit the model, we can evaluate performance on our test data using the test() method of our trainer.

In [21]:
hit_trainer.test(hit_module, datamodule=hit_dm)

Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 87.04it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss             104098.5546875
        test_mae            229.50115966796875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 104098.5546875, 'test_mae': 229.50115966796875}]

We can see that MAE has reduced to 229.50 compared to the 259 and 237 we had earlier for OLS and Lasso Regression and it's a quite a better result.

Now earlier we had made a CSV logger to log the data and errors.Now that nicely gives the logs in the form of a csv file that can be loaded with pandas dataframe.We can use that to make a few meaningful plots.